In [1]:
%reset -f

TO DO:
- reference
- find baseline hyperparameters (SVM linear or nonlinear?)
- find github of their work to see hyperparameters
- CLEAR MEMORY ON C DRIVE

In [2]:
import numpy as np
import pandas as pd
import string
import csv
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC 

In [3]:
#Extract data
# df = pd.read_csv('../Project_data/yelp_review_full_csv/yelp_data.csv', header = None) #contains both train and test data!
# df = df_orig.copy(deep=True)
# df = pd.read_csv('../Project_data/yelp_review_full_csv/test.csv', header = None) #contains both train and test data!

file = open('../Project_data/yelp_review_full_csv/yelp_data.csv', mode = 'r')
lines = file.readlines()
processed_data = []

#super slow (avoid)
# with open('../Project_data/yelp_review_full_csv/train.csv', newline='') as csvfile:
#     data = list(csv.reader(csvfile))
    
# print(data)

In [255]:
lines[1213]

'2,"I am a pretty easygoing guy. I would\'ve overlooked the fact(s) that:\\n\\n1. Jim Beam is your only bourbon. \\n2. For some reason two drinks came out to $9.90.* \\n3. Even with only 10 people in the bar, the music was absolutely deafening. \\n4. You charge a $3 cover for no discernible reason.\\n5. Rolling blunts is apparently kosher inside the bar, but sharing, not so much.\\n\\nHowever, I can\'t let this slide: your claw machine is BULLSHIT. That is the weakest claw of all time, and there\'s no chance it is EVER picking up anything in that cage. I feel violated, and I want my dollar back. \\n___________\\n\\n*I think that\'s some sort of weird PA tax thing."\n'

In [256]:
lines[1213][0:1] #get label

'2'

In [257]:
lines[1213][:-1] #remove \n at the end

'2,"I am a pretty easygoing guy. I would\'ve overlooked the fact(s) that:\\n\\n1. Jim Beam is your only bourbon. \\n2. For some reason two drinks came out to $9.90.* \\n3. Even with only 10 people in the bar, the music was absolutely deafening. \\n4. You charge a $3 cover for no discernible reason.\\n5. Rolling blunts is apparently kosher inside the bar, but sharing, not so much.\\n\\nHowever, I can\'t let this slide: your claw machine is BULLSHIT. That is the weakest claw of all time, and there\'s no chance it is EVER picking up anything in that cage. I feel violated, and I want my dollar back. \\n___________\\n\\n*I think that\'s some sort of weird PA tax thing."'

In [258]:
lines[1213][2:-1] #get text review

'"I am a pretty easygoing guy. I would\'ve overlooked the fact(s) that:\\n\\n1. Jim Beam is your only bourbon. \\n2. For some reason two drinks came out to $9.90.* \\n3. Even with only 10 people in the bar, the music was absolutely deafening. \\n4. You charge a $3 cover for no discernible reason.\\n5. Rolling blunts is apparently kosher inside the bar, but sharing, not so much.\\n\\nHowever, I can\'t let this slide: your claw machine is BULLSHIT. That is the weakest claw of all time, and there\'s no chance it is EVER picking up anything in that cage. I feel violated, and I want my dollar back. \\n___________\\n\\n*I think that\'s some sort of weird PA tax thing."'

preprocessing

In [4]:
target = [] #target vector for entire set

for line in lines:
    review_target = line[0:1]
    line = line[2:-1]
    line = line.replace("<br", "")
    line = line.replace("\\n", "")
    line = line.replace("\n", "")
    line = line.translate(str.maketrans('', '', string.punctuation)) #remove punctutaion
    processed_data.append(line)
    target.append(review_target)

target = list(map(int, target)) #convert from string to int

In [260]:
processed_data[1213]

'I am a pretty easygoing guy I wouldve overlooked the facts that1 Jim Beam is your only bourbon 2 For some reason two drinks came out to 990 3 Even with only 10 people in the bar the music was absolutely deafening 4 You charge a 3 cover for no discernible reason5 Rolling blunts is apparently kosher inside the bar but sharing not so muchHowever I cant let this slide your claw machine is BULLSHIT That is the weakest claw of all time and theres no chance it is EVER picking up anything in that cage I feel violated and I want my dollar back I think thats some sort of weird PA tax thing'

In [261]:
target

[5,
 2,
 4,
 4,
 1,
 5,
 5,
 1,
 2,
 3,
 1,
 1,
 4,
 2,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 3,
 2,
 5,
 1,
 1,
 1,
 2,
 2,
 4,
 2,
 2,
 4,
 3,
 5,
 4,
 5,
 3,
 5,
 4,
 5,
 5,
 5,
 5,
 2,
 3,
 5,
 1,
 4,
 2,
 1,
 3,
 1,
 5,
 3,
 3,
 1,
 4,
 1,
 1,
 3,
 1,
 2,
 2,
 1,
 5,
 4,
 2,
 1,
 5,
 3,
 5,
 4,
 1,
 4,
 4,
 4,
 2,
 4,
 2,
 3,
 2,
 1,
 2,
 2,
 4,
 1,
 3,
 5,
 4,
 5,
 1,
 5,
 5,
 2,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 2,
 3,
 2,
 3,
 5,
 2,
 3,
 2,
 5,
 2,
 3,
 2,
 3,
 1,
 1,
 2,
 2,
 2,
 3,
 1,
 1,
 2,
 2,
 3,
 4,
 2,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 2,
 1,
 3,
 2,
 2,
 4,
 4,
 5,
 2,
 2,
 3,
 5,
 2,
 3,
 5,
 5,
 2,
 5,
 4,
 3,
 2,
 1,
 3,
 3,
 2,
 3,
 1,
 1,
 3,
 3,
 4,
 5,
 2,
 2,
 1,
 3,
 2,
 2,
 3,
 4,
 2,
 1,
 2,
 5,
 2,
 3,
 3,
 1,
 2,
 1,
 3,
 3,
 3,
 1,
 2,
 1,
 3,
 2,
 2,
 1,
 4,
 5,
 2,
 3,
 1,
 2,
 3,
 1,
 3,
 2,
 3,
 4,
 2,
 3,
 2,
 1,
 3,
 4,
 2,
 3,
 4,
 4,
 2,
 3,
 4,
 2,
 2,
 3,
 4,
 5,
 4,
 1,
 3,
 4,
 2,
 4,
 1,
 5,
 4,
 4,
 2,
 2,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,


In [10]:
# np.random.seed(42) #ensure same split everytime
# train, valid, test = np.split(df.sample(frac=1), [int(.8*len(df)), int(.9*len(df))]) #0.8 = 80% for train and (1-0.9)=10% for valid

# train=df.sample(frac=0.8,random_state=200)
# test=df.drop(train.index)

X_train, X_remain, y_train, y_remain = train_test_split(processed_data[0:335000], target[0:335000], test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_remain, y_remain, test_size=0.5, random_state=42)

In [210]:
# del processed_data
# del X_remain
# del y_remain

Vectorizing. Doing Frequency bag-of-words representation first

In [11]:
stop_words = text.ENGLISH_STOP_WORDS #Remove stop words
# vectorizer = CountVectorizer(analyzer = "word", binary = False, lowercase=True, ngram_range = (2,2), tokenizer = None, preprocessor = None, stop_words = stop_words, max_features = 15000)
# vectorizer = CountVectorizer(analyzer = "word", binary = False, lowercase=True, tokenizer = None, preprocessor = None, max_features = 20000) #best with 0.13 and 100,000 (57.54%)
vectorizer = CountVectorizer(analyzer = "word", binary = True, lowercase=True, tokenizer = None, preprocessor = None, max_features = 10000)

#notes:
#max features and data = 20,000 and 125000

train_data_features = vectorizer.fit_transform(X_train)
valid_data_features = vectorizer.transform(X_valid)
test_data_features = vectorizer.transform(X_test)

#Normalization
normalizer_train = Normalizer().fit(X=train_data_features)
train_data_features = normalizer_train.transform(train_data_features)

normalizer_valid = Normalizer().fit(X=valid_data_features)
valid_data_features = normalizer_valid.transform(valid_data_features)

normalizer_test = Normalizer().fit(X=test_data_features)
test_data_features = normalizer_test.transform(test_data_features)

Feature list

In [316]:
vectorizer.get_feature_names()

['10 bucks',
 '10 days',
 '10 min',
 '10 mins',
 '10 minute',
 '10 minutes',
 '10 people',
 '10 pm',
 '10 times',
 '10 years',
 '1015 minutes',
 '12 hour',
 '12 hours',
 '12 stars',
 '12 years',
 '15 hours',
 '15 min',
 '15 mins',
 '15 minute',
 '15 minutes',
 '15 years',
 '1520 minutes',
 '1st time',
 '20 bucks',
 '20 min',
 '20 mins',
 '20 minute',
 '20 minutes',
 '20 people',
 '20 years',
 '24 hour',
 '24 hours',
 '25 minutes',
 '25 stars',
 '25 years',
 '2nd time',
 '30 days',
 '30 min',
 '30 mins',
 '30 minute',
 '30 minutes',
 '30 seconds',
 '30 years',
 '35 minutes',
 '35 stars',
 '3rd time',
 '40 minutes',
 '45 min',
 '45 mins',
 '45 minute',
 '45 minutes',
 '45 stars',
 '50 cents',
 '50 minutes',
 '510 minutes',
 '90 minutes',
 'able eat',
 'able make',
 'able order',
 'absolute favorite',
 'absolute worst',
 'absolutely amazing',
 'absolutely delicious',
 'absolutely horrible',
 'absolutely love',
 'absolutely loved',
 'absolutely terrible',
 'ac unit',
 'act like',
 'acted l

Getting list of frequency of bigrams (normalization included)

In [284]:
freqs = zip(vectorizer.get_feature_names(), train_data_features.sum(axis=0).tolist()[0])   #ONLY FOR FREQ BAG OF WORDS 
# sort from largest to smallest
word_freqs = sorted(freqs, key=lambda x: -x[1])

In [285]:
word_freqs

[('food', 5918.696260728457),
 ('place', 5631.282527347432),
 ('good', 5584.075155576131),
 ('like', 3996.161796630636),
 ('just', 3860.1930748554973),
 ('great', 3793.5654670891),
 ('service', 3783.29933962429),
 ('time', 3243.814313331383),
 ('really', 2576.729285973427),
 ('dont', 2344.681725322747),
 ('got', 1825.1410948082118),
 ('nice', 1771.49080770499),
 ('im', 1720.4563406023753),
 ('ive', 1715.3772997857504),
 ('best', 1607.8643785588506),
 ('didnt', 1601.3117205863853),
 ('staff', 1552.545370192327),
 ('went', 1535.5456440553085),
 ('order', 1527.7846520166793),
 ('little', 1523.8228598238006),
 ('love', 1520.1035937090817),
 ('people', 1482.891715623911),
 ('pretty', 1476.65714167727),
 ('better', 1469.5475651591892),
 ('chicken', 1440.1048163032208),
 ('ordered', 1437.3140194032017),
 ('did', 1420.2668317115008),
 ('going', 1376.5816431294488),
 ('restaurant', 1350.6973785274463),
 ('friendly', 1344.2442578488751),
 ('know', 1340.7974054157771),
 ('came', 1290.806587765150

In [286]:
# del X_train
# del X_test
# del X_valid

In [431]:
penalty = np.linspace(0.01,1,num = 20)
acc_arr = np.zeros([2,np.size(penalty)])
acc_arr[0,:] = penalty

acc_arr

array([[0.01      , 0.06210526, 0.11421053, 0.16631579, 0.21842105,
        0.27052632, 0.32263158, 0.37473684, 0.42684211, 0.47894737,
        0.53105263, 0.58315789, 0.63526316, 0.68736842, 0.73947368,
        0.79157895, 0.84368421, 0.89578947, 0.94789474, 1.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

In [12]:
# clf = SVC(kernel='linear', C = 0.13)   
clf = LinearSVC(C = 0.13) 
clf.fit(train_data_features.toarray(), y_train) #train classifier
valid_predicted = clf.predict(valid_data_features.toarray()) #Predict from test input vectors
valid_predicted = list(map(int, valid_predicted)) #convert from string to int
test_predicted = clf.predict(test_data_features.toarray()) #Predict from test input vectors
test_predicted = list(map(int, test_predicted)) #convert from string to int

valid_accuracy = accuracy_score(y_valid, valid_predicted)
print("validation accuracy = ", valid_accuracy)
# test_accuracy = accuracy_score(y_test, test_predicted)
# print("test accuracy = ", test_accuracy)

validation accuracy =  0.5708059701492537


In [428]:
penalty = np.linspace(0.01,1,num = 20)
acc_arr = np.zeros([2,np.size(penalty)])
acc_arr[0,:] = penalty

for idx, p in enumerate(penalty):
    # clf = MultinomialNB(alpha=0.05) 
    clf = LinearSVC(C = p) 
    clf.fit(train_data_features.toarray(), y_train) #train classifier
    valid_predicted = clf.predict(valid_data_features.toarray()) #Predict from test input vectors
    valid_predicted = list(map(int, valid_predicted)) #convert from string to int
#     test_predicted = clf.predict(test_data_features.toarray()) #Predict from test input vectors
#     test_predicted = list(map(int, test_predicted)) #convert from string to int

    acc = accuracy_score(y_valid, valid_predicted)
    acc_arr[1, idx] = acc

# valid_accuracy = accuracy_score(y_valid, valid_predicted)
# print("validation accuracy = ", valid_accuracy)
# test_accuracy = accuracy_score(y_test, test_predicted)
# print("test accuracy = ", test_accuracy)

In [430]:
acc_arr.transpose()

array([[0.01      , 0.5428    ],
       [0.05061224, 0.5681    ],
       [0.09122449, 0.5749    ],
       [0.13183673, 0.5754    ],
       [0.17244898, 0.5754    ],
       [0.21306122, 0.5754    ],
       [0.25367347, 0.5755    ],
       [0.29428571, 0.5751    ],
       [0.33489796, 0.575     ],
       [0.3755102 , 0.5747    ],
       [0.41612245, 0.574     ],
       [0.45673469, 0.5728    ],
       [0.49734694, 0.5727    ],
       [0.53795918, 0.5726    ],
       [0.57857143, 0.5736    ],
       [0.61918367, 0.5737    ],
       [0.65979592, 0.5732    ],
       [0.70040816, 0.5728    ],
       [0.74102041, 0.5731    ],
       [0.78163265, 0.573     ],
       [0.8222449 , 0.5735    ],
       [0.86285714, 0.5735    ],
       [0.90346939, 0.5737    ],
       [0.94408163, 0.5733    ],
       [0.98469388, 0.5735    ],
       [1.02530612, 0.5734    ],
       [1.06591837, 0.5728    ],
       [1.10653061, 0.5726    ],
       [1.14714286, 0.572     ],
       [1.1877551 , 0.5713    ],
       [1.